In [1]:
# _*_ coding: utf-8 _*_
# @Time: 2022/01/21 14:10 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 南方医院他克莫司：1. 数据处理

# 原始数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\Tcms_southern_hospital


## 导入预定义函数

In [3]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d-%m-%Y %H:%M:%S")
        return a
    except:
        return np.NaN

In [4]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [5]:
# 过滤特殊字符
def filter_spec_character(df,feature):
    df[feature]=df[feature].apply(lambda x: re.sub('<|>|\+|\/|-|cm|kg','',str(x)))
    return df

In [6]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else 100*median_value)
    df=df[df[feature].notnull()]
    return df

## 读取doctor_order数据库表

In [7]:
# 从df_doctor_order读入数据
df_doctor_order =pd.read_csv(project_path +'/data/raw_data/df_doctor_order.csv')
if 'Unnamed: 0' in df_doctor_order.columns:
    df_doctor_order = df_doctor_order.drop(['Unnamed: 0'], axis=1)

In [8]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(1130347, 30)
1079


In [9]:
# 提取有效字段
df_doctor_order=df_doctor_order[['patient_id','case_no','long_d_order','drug_name','drug_spec','dosage','frequency',
                                 'start_datetime','end_datetime','statusdesc']]

In [10]:
df_doctor_order.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,7365906,3721037,0,"肾功五项（UREA,CR,UA,TCO2,CysC）",NaN,NaN,ONCE,2018-06-07 17:37:52,NaN,执行
1,7365906,3721037,0,血型鉴定(ABO+RhD),NaN,NaN,ONCE,2018-06-07 17:37:53,NaN,执行
2,7365906,3721037,0,5%葡萄糖注射液[250ml*1袋],250ml*1袋,250ml,ONCE,2018-06-08 09:41:02,NaN,核实
3,7365906,3721037,0,脂溶性维生素(Ⅱ)/水溶性维生素组合包装[co*1盒],co*1盒,1co,ONCE,2018-06-08 09:41:02,NaN,核实
4,7365906,3721037,0,静脉输液第二组起每组,NaN,NaN,ONCE,2018-06-08 09:41:02,NaN,核实


In [11]:
# 删除重复数据
df_doctor_order=df_doctor_order.drop_duplicates(['case_no','long_d_order','drug_name','dosage','frequency','start_datetime','end_datetime','statusdesc']
                                                ,keep='first')

In [12]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(977887, 10)
1079


## 读取diagnostic_record

In [13]:
# 从diagnose_content读入数据
df_diagnostic_record =pd.read_csv(project_path +'/data/raw_data/df_diagnostic.csv')
if 'Unnamed: 0' in df_diagnostic_record.columns:
    df_diagnostic_record = df_diagnostic_record.drop(['Unnamed: 0'], axis=1)

In [14]:
print(df_diagnostic_record.shape)
print(df_diagnostic_record['patient_id'].nunique())

(88774, 17)
8508


In [15]:
#用来判断某列是否有缺失值
df_diagnostic_record.isnull().any() 

diagnostic_record_id    False
patient_id              False
inp_record_id            True
outp_record_id           True
record_date             False
diagnostic_type         False
diagnostic_content      False
treatment_days           True
treatment_result         True
IsPrimary                True
case_no                  True
diagnosis_code           True
outp_no                  True
create_time             False
update_time             False
migrate_time            False
diagnostic_name          True
dtype: bool

In [16]:
# 提取诊断有效字段
df_diagnostic_record=df_diagnostic_record[['patient_id','case_no','record_date','diagnostic_content']]
# 查看重复数据
df_diagnostic_record[df_diagnostic_record.duplicated(subset=['patient_id','case_no','record_date','diagnostic_content'],keep='first')]

,patient_id,case_no,record_date,diagnostic_content
816,6554943,NaN,2016-12-25 20:33:28,测试
819,6554943,NaN,2016-12-25 20:33:35,测试
822,6554943,NaN,2016-12-25 20:33:38,测试
827,6554943,NaN,2016-12-25 20:33:43,测试
874,6554943,NaN,2016-12-25 20:55:00,测试
...,...,...,...,...
84635,8283895,NaN,2020-06-19 11:06:36,膜增生性肾小球肾炎
87034,5836499,NaN,2020-07-14 08:10:33,肾移植状态
87035,5836499,NaN,2020-07-14 08:10:33,肾移植状态
87659,7503824,NaN,2020-07-20 09:22:10,肾移植状态


In [17]:
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_diagnostic_record.xlsx')
df_diagnostic_record.to_excel(writer)
writer.save()

## 读取test_result数据表

### test_record

In [18]:
# 从df_test_record读入数据
df_test_record =pd.read_csv(project_path +'/data/raw_data/df_test_record.csv')
if 'Unnamed: 0' in df_test_record.columns:
    df_test_record = df_test_record.drop(['Unnamed: 0'], axis=1)

In [19]:
print(df_test_record.shape)
print(df_test_record['patient_id'].nunique())

(361590, 29)
3193


In [20]:
# 提取test_record有效字段
df_test_record=df_test_record[['test_record_id','patient_id','case_no','test_date']]

In [21]:
df_test_record.head()

,test_record_id,patient_id,case_no,test_date
0,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00
1,1000009730||B006||1,4980556,NaN,2016-11-28 09:18:00
2,1000009730||B008||1,4980556,NaN,2016-11-28 09:18:00
3,1000009730||B010||1,4980556,NaN,2016-11-28 09:18:00
4,1000009730||B034||1,4980556,NaN,2016-11-28 09:18:00


In [22]:
#用来判断某列是否有缺失值
df_test_record.isnull().any() 

test_record_id    False
patient_id        False
case_no            True
test_date         False
dtype: bool

### test_result

In [23]:
# 从df_test_result读入数据
df_test_result =pd.read_csv(project_path +'/data/raw_data/df_result.csv')
if 'Unnamed: 0' in df_test_result.columns:
    df_test_result = df_test_result.drop(['Unnamed: 0'], axis=1)

In [24]:
print(df_test_result.shape)

(2741436, 15)


In [25]:
df_test_result=df_test_result[['test_record_id','project_name','test_result','result_unit','is_normal','refer_scope','synonym']]

In [26]:
#用来判断某列是否有缺失值
df_test_result.isnull().any() 

test_record_id    False
project_name      False
test_result        True
result_unit        True
is_normal          True
refer_scope        True
synonym            True
dtype: bool

In [27]:
# 删除检测项为空的数据
df_test_result=df_test_result[df_test_result['test_result'].notnull()]

In [28]:
# 查看重复数据
df_test_result[df_test_result.duplicated(subset=['test_record_id','project_name','test_result','synonym'],keep='first')]

,test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym
59755,800000127238||K206||1,ABO血型鉴定,A,NaN,NaN,NaN,NaN
205243,800000487995||K206||1,ABO血型鉴定,B,NaN,NaN,NaN,NaN
578049,800001349716||K206||1,ABO血型鉴定,正定A型，反定O型,NaN,N,NaN,NaN
583858,800001370023||K206||1,ABO血型鉴定,A,NaN,NaN,NaN,NaN
672691,800001580097||K206||1,ABO血型鉴定,A,NaN,NaN,NaN,NaN
821461,800002061283||K206||1,ABO血型鉴定,正定O型，反定B型,NaN,N,NaN,NaN
873427,800002247110||K206||1,ABO血型鉴定,O,NaN,NaN,NaN,NaN
900629,800002346396||K206||1,ABO血型鉴定,A,NaN,NaN,NaN,NaN
936592,800002477327||K206||1,ABO血型鉴定,A,NaN,NaN,NaN,NaN
937809,800002480042||K206||1,ABO血型鉴定,正定B型，反定AB型,NaN,N,NaN,NaN


In [29]:
# 删除test_result数据
df_test_result=df_test_result.drop_duplicates(subset=['test_record_id','project_name','test_result'],keep='first')
df_test_result=df_test_result.reset_index(drop=True)

In [30]:
print(df_test_result.shape)

(2741009, 7)


In [31]:
# 删除<>号
df_test_result=filter_spec_character(df_test_result,'test_result')

In [32]:
print(df_test_result.shape)

(2741009, 7)


### test_record+test_result

In [33]:
df_test=pd.merge(df_test_record,df_test_result,on=['test_record_id'],how='inner')

In [34]:
df_test.head()

,test_record_id,patient_id,case_no,test_date,project_name,test_result,result_unit,is_normal,refer_scope,synonym
0,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00,丙氨酸氨基转移酶,10,U/L,N,7-40,ALT
1,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00,天门冬氨酸氨基转移酶,13,U/L,N,13-35,AST
2,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00,转氨酶比值,1.3,NaN,N,1.0-1.5,AST/ALT
3,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00,总蛋白,72.3,g/L,N,65.0-85.0,TP
4,1000009730||B005||1,4980556,NaN,2016-11-28 09:18:00,白蛋白,43.7,g/L,N,40.0-55.0,ALB


In [35]:
print(df_test.shape)
print(df_test['patient_id'].nunique())

(2741009, 10)
3193


## 提取人口学特征

### gender

In [36]:
# 从patient_info提取gender,race,birth_year
df_patient_info =pd.read_csv(project_path +'/data/raw_data/df_patient_info.csv')
if 'Unnamed: 0' in df_patient_info.columns:
    df_patient_info = df_patient_info.drop(['Unnamed: 0'], axis=1)


In [37]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(8510, 24)
8510


In [38]:
df_gender=df_patient_info[['patient_id','gender']]
df_gender['gender']=df_gender['gender'].apply(lambda x: 1 if re.match('男',x) else 0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
#用来判断某列是否有缺失值
df_gender.isnull().any() 

patient_id    False
gender        False
dtype: bool

In [40]:
# 查看重复数据
df_gender[df_gender.duplicated(subset=['patient_id','gender'],keep='first')]

,patient_id,gender


In [41]:
df_gender.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_gender.xlsx')

### age

In [42]:
# 从patient_info提取gender,race,birth_year
df_inp_record =pd.read_csv(project_path +'/data/raw_data/inp_record.csv')
if 'Unnamed: 0' in df_inp_record.columns:
    df_inp_record = df_inp_record.drop(['Unnamed: 0'], axis=1)

In [43]:
df_age=df_inp_record[['patient_id','case_no','age']]

In [44]:
print(df_age.shape)
print(df_age['patient_id'].nunique())

(1759, 3)
1079


In [45]:
df_age

,patient_id,case_no,age
0,7365906,3721037,39
1,539401,3811092,11
2,76038,3818984,17
3,307084,3826880,7
4,6051695,3828457,45
...,...,...,...
1754,8262445,9842063,32
1755,7891340,9845643,3
1756,8165400,9846289,57
1757,7762810,9847836,33


In [46]:
df_age.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_age.xlsx')

### height

In [47]:
# 从patient_sign_record提取height、weight
df_patient_sign_record=pd.read_csv(project_path +'/data/raw_data/patient_vital.csv')
if 'Unnamed: 0' in df_patient_sign_record.columns:
    df_patient_sign_record = df_patient_sign_record.drop(['Unnamed: 0'], axis=1)
df_patient_sign_record=df_patient_sign_record[['patient_id','case_no','record_date','sign_type','record_content','record_unit']]

In [48]:
df_patient_sign_record.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_sign_record.xlsx')

In [49]:
# 提取身高信息
df_height=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('身高')]

In [50]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(1964, 6)
1079


In [51]:
df_height

,patient_id,case_no,record_date,sign_type,record_content,record_unit
0,7365906,3721037,2018-06-07 14:00:00,身高(cm),169,cm
152,539401,3811092,2018-06-20 14:00:00,身高(cm),130,cm
274,76038,3818984,2018-06-21 14:00:00,身高(cm),147,cm
860,76038,3818984,2018-07-30 14:00:00,身高(cm),147,cm
1326,307084,3826880,2018-06-22 14:00:00,身高(cm),120,cm
...,...,...,...,...,...,...
342227,8262445,9842063,2020-07-30 10:00:00,身高(cm),164,cm
342230,7891340,9845643,2020-07-30 14:00:00,身高(cm),92,cm
342275,8165400,9846289,2020-07-30 14:00:00,身高(cm),162,cm
342297,7762810,9847836,2020-07-30 14:00:00,身高(cm),174,cm


In [52]:
# 过滤文字
df_height=filter_string(df_height,'record_content')
# 过滤特殊字符
df_height=filter_spec_character(df_height,'record_content')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(1834, 6)
1072


In [54]:
# 查看重复数据
df_height[df_height.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
860,76038,3818984,2018-07-30 14:00:00,身高(cm),147,cm
3459,6681563,3947252,2018-07-15 14:00:00,身高(cm),165,cm
5882,1148109,3975434,2018-09-11 14:00:00,身高(cm),109,cm
6418,1148109,3975434,2018-10-02 14:00:00,身高(cm),109,cm
7602,7441638,3987166,2018-07-19 14:00:00,身高(cm),178,cm
...,...,...,...,...,...,...
336533,8037660,9585336,2020-07-06 14:00:00,身高(cm),92,cm
338579,6329653,9671334,2020-07-13 14:00:00,身高(cm),120,cm
339899,6973480,9714809,2020-07-23 14:00:00,身高(cm),165,cm
341585,5990104,9808390,2020-07-27 10:00:00,身高(cm),159,cm


In [55]:
# 排序
df_height=df_height.sort_values(by=['case_no','record_date'],ascending=[True,True])
# 删除重复数据
df_height=df_height.drop_duplicates(subset=['case_no','record_content'],keep='last')

In [56]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(1735, 6)
1072


In [57]:
df_height.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_height.xlsx')

### weight

In [58]:
# 提取体重信息
df_weight=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('体重')]

In [59]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(11106, 6)
1079


In [60]:
df_weight.head()

,patient_id,case_no,record_date,sign_type,record_content,record_unit
60,7365906,3721037,2018-06-14 14:00:00,体重(kg),39,kg
63,7365906,3721037,2018-06-21 14:00:00,体重(kg),39,kg
69,7365906,3721037,2018-06-07 14:00:00,体重(kg),39.5,kg
184,539401,3811092,2018-07-09 14:00:00,体重(kg),25,kg
191,539401,3811092,2018-07-02 14:00:00,体重(kg),25,kg


In [61]:
# 过滤文字
df_weight=filter_string(df_weight,'record_content')

In [62]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(10292, 6)
1075


In [63]:
'224'.replace('..','.')

'224'

In [64]:
# 过滤异常值
df_weight=df_weight[~ df_weight['record_content'].str.contains('0')]
# 过滤特殊字符
df_weight=filter_spec_character(df_weight,'record_content')
df_weight['record_content']=df_weight['record_content'].apply(lambda x: str(x).replace('..','.'))

In [65]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(8794, 6)
996


In [66]:
# 查看重复数据
df_weight[df_weight.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
63,7365906,3721037,2018-06-21 14:00:00,体重(kg),39,kg
191,539401,3811092,2018-07-02 14:00:00,体重(kg),25,kg
202,539401,3811092,2018-06-20 14:00:00,体重(kg),25,kg
233,539401,3811092,2018-06-25 10:00:00,体重(kg),25,kg
864,76038,3818984,2018-07-30 14:00:00,体重(kg),31,kg
...,...,...,...,...,...,...
340394,8166984,9741221,2020-07-19 14:00:00,体重(kg),56,kg
341323,6277929,9782516,2020-07-27 10:00:00,体重(kg),57,kg
341385,6277929,9782516,2020-07-30 14:00:00,体重(kg),57,kg
341586,5990104,9808390,2020-07-27 10:00:00,体重(kg),45,kg


In [67]:
# 排序
df_weight=df_weight.sort_values(by=['case_no','record_date'],ascending=[True,True])
# 删除重复数据
df_weight=df_weight.drop_duplicates(subset=['case_no','record_content'],keep='last')

In [68]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(5016, 6)
996


In [69]:
df_weight.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_weight.xlsx')

In [70]:
df_weight.head()

,patient_id,case_no,record_date,sign_type,record_content,record_unit
69,7365906,3721037,2018-06-07 14:00:00,体重(kg),39.5,kg
63,7365906,3721037,2018-06-21 14:00:00,体重(kg),39,kg
184,539401,3811092,2018-07-09 14:00:00,体重(kg),25,kg
864,76038,3818984,2018-07-30 14:00:00,体重(kg),31,kg
1325,76038,3818984,2018-08-20 14:00:00,体重(kg),28.5,kg


# 纳排

## 纳入标准

### 提取他克莫司用药

In [71]:
# 提取他克莫司用药
df_Tcms = df_doctor_order[df_doctor_order['drug_name'].str.contains('他克莫司') & (df_doctor_order['drug_name'].str.contains('胶囊'))]

In [72]:
# 排序
df_Tcms=df_Tcms.sort_values(['patient_id','case_no','start_datetime'])

In [73]:
print(df_Tcms.shape)
print(df_Tcms['patient_id'].nunique())

(5670, 10)
821


In [75]:
# 保存丙戊酸用药数据
df_Tcms.to_excel(project_path+'/data/processed_data/df_2.1.1_他克莫司用药.xlsx')

### 系统性红斑狼疮等

In [76]:
# 从diagnostic_record中提取狼疮性肾炎、系统性红斑狼疮、肾病综合征
df_SLE=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('狼疮性肾炎|狼疮肾炎|系统性红斑狼疮|肾病综合')]

In [77]:
df_SLE['diagnostic_content'].unique()

array(['系统性红斑狼疮', '狼疮性肾炎', '肾病综合征', '肾病综合征，膜性肾病', '肾病综合征，肾小球轻微病变',
       '肾病综合征，局灶性节段性肾小球肾炎', '系统性红斑狼疮，累及器官或系统', '肾病综合征，特指病理', '系统性红斑狼疮?',
       '难治性肾病综合征', '肾病综合征激素依赖型', 'SLE，狼疮肾炎性，肾病综合征，急性肾损伤，低蛋白血症，高尿酸血症，',
       '肾病综合征复发', '肾病综合征伴节段硬化性肾小球肾炎', '肾病综合征，膜增殖性肾小球肾炎Ⅰ型',
       '肾病综合征伴有局灶性和节段性肾小球损害', '肾病综合症伴局灶硬化性肾小球肾', '肾病综合征,其他的',
       '肾病综合征，膜增殖性肾小球肾炎Ⅲ型', '难治性肾病综合征，激素耐药型', '肾病综合征伴有轻微的肾小球异常',
       '肾病综合征，难治性', '肾病综合征伴微小病变性肾小球肾炎', '系统性红斑狼疮，其他形式的',
       '难治性肾病综合征，局灶节段性肾小球硬化症', '妊娠合并系统性红斑狼疮', '肾病综合征伴微小病变性', '狼疮肾炎',
       '1.发热、皮疹查因：过敏反应？感染？系统性红斑狼疮相关？', '肾病综合征,局灶节段性肾小球硬化症',
       '系统性红斑狼疮血液系统损害', '继发性肾病综合征', '原发性肾病综合征', '局灶节段性肾小球硬化症,肾病综合征',
       '狼疮性肾炎IV+V型', '先天性肾病综合征', '系统性红斑狼疮伴妊娠'], dtype=object)

In [78]:
print(df_SLE.shape)
print(df_SLE['patient_id'].nunique())

(5028, 4)
548


In [79]:
# 纳入双相情感障碍
df_Tcms_SLE=df_Tcms[df_Tcms['patient_id'].isin(df_SLE['patient_id'].unique())]
df_Tcms_SLE=df_Tcms_SLE.sort_values(['case_no','start_datetime']).reset_index(drop=True)

In [80]:
print(df_Tcms_SLE.shape)
print(df_Tcms_SLE['patient_id'].nunique())

(746, 10)
185


In [81]:
df_Tcms_SLE.to_excel(project_path+'/data/processed_data/df_2.1.2_性红斑狼疮过滤的tcms用药.xlsx')

In [82]:
# ### 年龄筛选：18-65岁

# df_age_adult=df_age[(df_age['age']>=18) & (df_age['age']<=65)]

# df_age_adult

# print(df_age_adult.shape)
# print(df_age_adult['patient_id'].nunique())

# # 纳入年龄限制
# df_Tcms_age=df_Tcms_BD[df_Tcms_BD['patient_id'].isin(df_age_adult['patient_id'].unique())]

# print(df_Tcms_age.shape)
# print(df_Tcms_age['patient_id'].nunique())

# df_Tcms_age.to_excel(project_path+'/data/processed_data/df_temp_Tcms_age.xlsx')

### tdm筛选

#### 提取他克莫司tdm检测

In [83]:
# 提取用药后不良反应发生前的他克莫司血药浓度检测
df_test_Tcms=df_test[df_test['project_name'].str.contains('他克莫司药物浓度监测')]
# 排序
df_test_Tcms=df_test_Tcms.sort_values(['patient_id','case_no','test_date'])
df_test_Tcms=df_test_Tcms.reset_index(drop=True)

In [84]:
print(df_test_Tcms.shape)
print(df_test_Tcms['patient_id'].nunique())

(25758, 10)
2480


In [85]:
df_test_Tcms.head()

,test_record_id,patient_id,case_no,test_date,project_name,test_result,result_unit,is_normal,refer_scope,synonym
0,800000058858||Y001||1,2026,NaN,2017-02-22 09:47:00,他克莫司药物浓度监测,7.10,ng/ml,N,NaN,FK506
1,800000138028||Y001||1,2026,NaN,2017-05-03 09:46:00,他克莫司药物浓度监测,8.90,ng/ml,N,NaN,FK506
2,800000372373||Y001||1,2026,NaN,2017-07-19 09:45:00,他克莫司药物浓度监测,6.00,ng/ml,N,NaN,FK506
3,800000640391||Y001||1,2026,NaN,2017-09-11 09:50:00,他克莫司药物浓度监测,8.70,ng/ml,N,NaN,FK506
4,800000817383||Y001||1,2026,NaN,2017-12-11 09:47:00,他克莫司药物浓度监测,5.40,ng/ml,N,NaN,FK506


In [86]:
df_test_Tcms.project_name.unique()

array(['他克莫司药物浓度监测'], dtype=object)

In [87]:
df_test_Tcms['tdm_date']=df_test_Tcms['test_date'].apply(lambda x: str(x).split(' ')[0])

In [88]:
df_test_Tcms['test_result']=df_test_Tcms['test_result'].astype('float')

In [89]:
df_test_Tcms.isnull().any()

test_record_id    False
patient_id        False
case_no            True
test_date         False
project_name      False
test_result       False
result_unit       False
is_normal         False
refer_scope        True
synonym           False
tdm_date          False
dtype: bool

In [90]:
df_test_Tcms=df_test_Tcms[df_test_Tcms.case_no.notnull()].reset_index(drop=True)

In [91]:
print(df_test_Tcms.shape)
print(df_test_Tcms['patient_id'].nunique())

(3779, 11)
744


In [92]:
# 提取患者的tdm检测
df_test_Tcms_choose=df_test_Tcms[df_test_Tcms['patient_id'].isin(df_Tcms_SLE.patient_id.unique())]
df_test_Tcms_choose=df_test_Tcms_choose.sort_values(['patient_id','test_date'])
df_test_Tcms_choose=df_test_Tcms_choose.reset_index(drop=True)

In [93]:
print(df_test_Tcms_choose.shape)
print(df_test_Tcms_choose['patient_id'].nunique())
print(df_test_Tcms_choose['case_no'].nunique())

(179, 11)
99
126


In [94]:
df_test_Tcms_choose.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   test_record_id  179 non-null    object 
 1   patient_id      179 non-null    int64  
 2   case_no         179 non-null    float64
 3   test_date       179 non-null    object 
 4   project_name    179 non-null    object 
 5   test_result     179 non-null    float64
 6   result_unit     179 non-null    object 
 7   is_normal       179 non-null    object 
 8   refer_scope     0 non-null      object 
 9   synonym         179 non-null    object 
 10  tdm_date        179 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 15.5+ KB


In [95]:
# 保存丙戊酸血药浓度检测
df_test_Tcms_choose.to_excel(project_path+'/data/processed_data/df_2.1.3.1_test_Tcms.xlsx')

In [96]:
'''
tdm很大，dosage很小，也可能删除
'''

'\ntdm很大，dosage很小，也可能删除\n'

In [97]:
# df_test_Tcms_choose=pd.read_excel(project_path+'/data/processed_data/df_2.1.3.1_人工调整test_Tcms.xlsx')
# if 'Unnamed: 0' in df_test_Tcms_choose.columns:
#     df_test_Tcms_choose = df_test_Tcms_choose.drop(['Unnamed: 0'], axis=1)

In [98]:
print(df_test_Tcms_choose.shape)
print(df_test_Tcms_choose['patient_id'].nunique())
print(df_test_Tcms_choose['case_no'].nunique())

(179, 11)
99
126


#### tdm筛选

In [99]:
# 合并他克莫司用药和tdm检测
df_Tcms_tdm_filter=df_Tcms_SLE[df_Tcms_SLE.case_no.isin(df_test_Tcms_choose.case_no.unique())]
# 排序
df_Tcms_tdm_filter=df_Tcms_tdm_filter.sort_values(['case_no','start_datetime']).reset_index(drop=True)

In [100]:
print(df_Tcms_tdm_filter.shape)
print(df_Tcms_tdm_filter['patient_id'].nunique())
print(df_Tcms_tdm_filter['case_no'].nunique())

(433, 10)
99
126


In [101]:
# 保存tdm筛选的他克莫司用药
df_Tcms_tdm_filter.to_excel(project_path+'/data/processed_data/df_2.1.3.2_tdm筛选的他克莫司用药.xlsx')

In [102]:
'''
手动合并同一天的多次用药记录，使之一天最多只有一条临时医嘱和长期医嘱
手动修改过高剂量。
tdm很大，dosage很小，也可能删除
last_dosage和dosage相差较大，删除
'''

'\n手动合并同一天的多次用药记录，使之一天最多只有一条临时医嘱和长期医嘱\n手动修改过高剂量。\ntdm很大，dosage很小，也可能删除\nlast_dosage和dosage相差较大，删除\n'

In [103]:
df_Tcms_tdm_filter=pd.read_excel(project_path+'/data/processed_data/df_2.1.3.2_人工处理的tdm筛选的他克莫司用药_剂量调低.xlsx')
if 'Unnamed: 0' in df_Tcms_tdm_filter.columns:
    df_Tcms_tdm_filter = df_Tcms_tdm_filter.drop(['Unnamed: 0'], axis=1)

In [104]:
print(df_Tcms_tdm_filter.shape)
print(df_Tcms_tdm_filter['patient_id'].nunique())
print(df_Tcms_tdm_filter['case_no'].nunique())

(334, 10)
99
126


### 拆分长期医嘱和临时医嘱

#### 计算他克莫司日剂量

In [105]:
df_Tcms_dosage=df_Tcms_tdm_filter.copy()
# 清洗单次日剂量dosage
# 1)'粒'根据剂型转换为mg数值
for i in range(df_Tcms_dosage.shape[0]):
    if '粒' in df_Tcms_dosage.loc[i, 'dosage']:
        num = df_Tcms_dosage.loc[i, 'dosage'].split('粒')[0]
        one_w = df_Tcms_dosage.loc[i, 'drug_spec'].split('mg')[0]
        df_Tcms_dosage.loc[i, 'dosage'] = str(float(one_w) * int(num))

In [106]:
df_Tcms_dosage['dosage']=df_Tcms_dosage['dosage'].apply(lambda x: float(x.replace('mg', '')) if 'mg' in x else x)

In [107]:
df_Tcms_dosage.frequency.unique()

array(['2/日', '1/12小时', 'ONCE', '1/日', '1/早', '1/晚', '2/日(餐前)'],
      dtype=object)

In [108]:
# 清洗频次
one=['1/午','ONCE','1/日','1/日(餐前)','1/早','1/晚']
two=['1/12小时','2/日','2/日(餐前)']
three=['Tid','3/日（餐前）']
df_Tcms_dosage['frequency']=df_Tcms_dosage['frequency'].apply(lambda x: 1 if x in one else
                                                                2 if x in two else
                                                                3 if x in three else x)

In [109]:
# 计算日剂量
df_Tcms_dosage['日剂量']=df_Tcms_dosage['dosage'].astype('float') * df_Tcms_dosage['frequency'].astype('float')

In [110]:
df_Tcms_dosage

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0
1,3106885,4054467,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-08-03 09:56:12,2018-08-06 11:17:32,停止,2.0
2,3106885,4054467,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-08-06 11:17:32,NaN,核实,2.0
3,7455055,4068971,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,1,2018-07-25 10:42:12,NaN,核实,1.0
4,7455055,4068971,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,2.5,1,2018-07-25 18:30:46,2018-08-01 09:17:58,停止,2.5
...,...,...,...,...,...,...,...,...,...,...,...
329,5623241,9671945,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2,2,2020-07-22 16:59:25,2020-07-25 10:00:00,停止,4.0
330,8059463,9745132,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,4.5,1,2020-07-20 10:12:23,2020-07-22 10:28:00,停止,4.5
331,8059463,9745132,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,4.5,1,2020-07-22 11:00:02,2020-07-24 11:03:56,停止,4.5
332,8059463,9745132,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,3,2,2020-07-24 11:03:56,NaN,核实,6.0


In [111]:
print(df_Tcms_dosage.shape)
print(df_Tcms_dosage['patient_id'].nunique())
print(df_Tcms_dosage['case_no'].nunique())

(334, 11)
99
126


In [112]:
df_Tcms_dosage.to_excel(project_path+'/data/processed_data/df_2.1.4.1_计算他克莫司日剂量.xlsx')

#### 提取用药日期

In [113]:
# 临时用药的结束时间=开始时间
aaa=df_Tcms_dosage[df_Tcms_dosage['end_datetime'].isnull()]
bbb=df_Tcms_dosage[df_Tcms_dosage['end_datetime'].notnull()]
aaa['end_datetime']=aaa['start_datetime']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [114]:
aaa

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
2,3106885,4054467,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-08-06 11:17:32,2018-08-06 11:17:32,核实,2.0
3,7455055,4068971,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,1,2018-07-25 10:42:12,2018-07-25 10:42:12,核实,1.0
6,7455055,4068971,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,1,2018-08-05 09:25:43,2018-08-05 09:25:43,核实,1.5
7,7455055,4068971,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,2,2,2018-08-06 17:46:03,2018-08-06 17:46:03,核实,4.0
9,6453342,4225762,0,他克莫司胶囊(福美欣)[1mg*50粒],1mg*50粒,1,1,2018-08-15 11:56:58,2018-08-15 11:56:58,核实,1.0
...,...,...,...,...,...,...,...,...,...,...,...
317,8091505,9641782,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-08 15:57:14,2020-07-08 15:57:14,核实,2.0
320,8329779,9662993,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,1,2020-07-10 16:27:56,2020-07-10 16:27:56,核实,1.0
324,5623241,9671945,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-07-12 13:29:01,2020-07-12 13:29:01,核实,5.0
328,5623241,9671945,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-07-22 08:43:29,2020-07-22 08:43:29,核实,5.0


In [115]:
df_Tcms_date=pd.concat([aaa,bbb],axis=0)
# 排序
df_Tcms_date=df_Tcms_date.sort_values(by=['patient_id','case_no','start_datetime'])
df_Tcms_date=df_Tcms_date.reset_index(drop=True)

In [116]:
print(df_Tcms_date.shape)
print(df_Tcms_date['patient_id'].nunique())
print(df_Tcms_date['case_no'].nunique())

(334, 11)
99
126


In [117]:
# 提取丙戊酸用药时间的年月日
df_Tcms_date['start_date']=df_Tcms_date['start_datetime'].apply(lambda x: str(x).split(' ')[0])
df_Tcms_date['end_date']=df_Tcms_date['end_datetime'].apply(lambda x: str(x).split(' ')[0])

In [118]:
df_Tcms_date.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
0,51620,5798617,1,他克莫司胶囊(弘盛)[0.5mg*50粒],0.5mg*50粒,2.5,1,2019-03-08 18:04:28,2019-03-08 18:06:31,停止,2.5,2019-03-08,2019-03-08
1,51620,5798617,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,2.5,1,2019-03-08 18:07:43,2019-03-13 11:05:59,停止,2.5,2019-03-08,2019-03-13
2,92856,4791498,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,2,2018-11-02 09:42:14,2018-11-08 12:28:00,停止,3.0,2018-11-02,2018-11-08
3,92856,4791498,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,2,2018-11-06 11:18:10,2018-11-06 11:18:10,核实,3.0,2018-11-06,2018-11-06
4,92856,8644797,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,2,2020-01-26 12:09:41,2020-02-01 09:18:57,停止,3.0,2020-01-26,2020-02-01


In [119]:
df_Tcms_date.to_excel(project_path+'/data/processed_data/df_2.1.3.2_提取年月日.xlsx')

#### 津源参考代码

In [120]:
# #对环孢素进行处理只包含注射液
# df = pd.read_excel(r'drug_2088_csa_injection_end.xls')

# df = df[['PATIENT_ID', 'DOSAGE_all','START_DATETIME', 'END_DATETIME']]
# df['PATIENT_ID'] = df['PATIENT_ID'].apply(lambda x: str(x))
# df['START_DATETIME'] = pd.to_datetime(df['START_DATETIME']).dt.strftime('%Y-%m-%d')
# df['END_DATETIME'] = pd.to_datetime(df['END_DATETIME']).dt.strftime('%Y-%m-%d')
# #%%
# import pandas as pd
# df['ll'] = df['DOSAGE_all']
# for i in range(len(df)):
#     df['ll'][i] = list(pd.date_range(start=df['START_DATETIME'][i], end=df['END_DATETIME'][i]))
    
# df['id_dose'] = df['PATIENT_ID'].map(str) + "_" + df['DOSAGE_all'].map(str)
# #%%
# df_1 = df[['id_dose','ll']]





# #%%
# def explode1(df, col):
#     dftem1 = pd.DataFrame()
#     for i in range(df.shape[0]):
#         dftem = df.iloc[i].to_frame().T
#         if isinstance(df.iloc[i][col], list):
#             if len(df.iloc[i][col]) > 0:
#                 for j in range(len(df.iloc[i][col])):
#                     dftem[col + "_"] = df.iloc[i][col][j]
#                     dftem1 = pd.concat([dftem1, dftem])
#             else:
#                 dftem1 = pd.concat([dftem1, dftem])
#         else:
#             dftem1 = pd.concat([dftem1, dftem])
#     del dftem1[col]
#     dftem1.rename(columns={col + "_": col}, inplace=True)
#     return dftem1

# #%%

# drug_2088_csa_injection_end_expand = explode1(df_1, 'll') 
# drug_2088_csa_injection_end_expand['PATIENT_ID'] = drug_2088_csa_injection_end_expand['id_dose'].apply(lambda x: x.split('_')[0])
# drug_2088_csa_injection_end_expand['DOSAGE'] = drug_2088_csa_injection_end_expand['id_dose'].apply(lambda x: x.split('_')[1])

# ll = drug_2088_csa_injection_end_expand
# drug_2088_csa_injection_end_expand1 = ll[[ 'PATIENT_ID', 'DOSAGE', 'll']].drop_duplicates(subset=['PATIENT_ID','ll'], keep='last')


#### 提取长期医嘱和临时医嘱

In [121]:
# 打长期医嘱标签
df_Tcms_date['long_d_order']=df_Tcms_date['statusdesc'].apply(lambda x: 1 if re.match('停止',x) else 0)

In [122]:
# 长期医嘱和临时医嘱分开
drug_temp = df_Tcms_date[df_Tcms_date['long_d_order']==0]
drug_long = df_Tcms_date[df_Tcms_date['long_d_order']==1]

In [123]:
print(drug_temp.shape)
print(drug_long.shape)

(133, 13)
(201, 13)


In [124]:
drug_temp.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
3,92856,4791498,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,2,2018-11-06 11:18:10,2018-11-06 11:18:10,核实,3.0,2018-11-06,2018-11-06
9,183176,5895566,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,1,2019-03-29 10:53:48,2019-03-29 10:53:48,核实,1.5,2019-03-29,2019-03-29
10,183176,8108818,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,1,2019-11-25 18:15:31,2019-11-25 18:15:31,核实,1.5,2019-11-25,2019-11-25
12,183176,8108818,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,1,2019-12-18 10:57:50,2019-12-18 10:57:50,核实,1.5,2019-12-18,2019-12-18
13,183176,8574883,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1.5,1,2020-01-14 21:36:49,2020-01-14 21:36:49,核实,1.5,2020-01-14,2020-01-14


In [125]:
# 查看重复数据
# drug_temp[drug_temp.duplicated(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])]

In [126]:
# 删除重复数据
drug_temp=drug_temp.drop_duplicates(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])
drug_long=drug_long.drop_duplicates(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])

In [127]:
print(drug_temp.shape)
print(drug_long.shape)

(133, 13)
(201, 13)


In [128]:
drug_temp[drug_temp['case_no']==8785699]

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
262,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-15 15:24:52,2020-03-15 15:24:52,核实,5.0,2020-03-15,2020-03-15
263,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-18 09:01:31,2020-03-18 09:01:31,核实,5.0,2020-03-18,2020-03-18
265,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-24 22:26:14,2020-03-24 22:26:14,核实,5.0,2020-03-24,2020-03-24
266,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-27 10:20:47,2020-03-27 10:20:47,核实,5.0,2020-03-27,2020-03-27
268,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-30 08:06:27,2020-03-30 08:06:27,核实,5.0,2020-03-30,2020-03-30


#### 处理临时医嘱

In [129]:
# 打临时时间标签，上午为0，下午为1
drug_temp['用药上下午标签']=drug_temp['start_datetime'].apply(lambda x: 0 if pd.to_datetime(x).hour <13 else 1)

In [130]:
drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')
drug_temp_result=drug_temp_all.copy()

In [131]:
# # ======= 单独对‘临时医嘱’处理 =======
# drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')
# drug_temp_1 = drug_temp.drop_duplicates(subset=['patient_id','start_date','用药上下午标签'],keep='first')
# drug_temp_group = drug_temp_1.groupby(['patient_id','start_date'])['日剂量'].sum().reset_index()   # 同一天的临时医嘱相加，保证每天一条临时医嘱

# # 拼接完整数据
# drug_temp_result = pd.merge(drug_temp_all.drop(columns=['日剂量','用药上下午标签']),drug_temp_group,on=['patient_id','start_date'],how='inner')

In [132]:
print(drug_temp_result.shape)

(133, 14)


In [133]:
# 计算临时医嘱的 日剂量， 根据开药时间
def dosage_temp(x):
    if (x['frequency']==2)&(x['小时_标签']>=13):
        return x['日剂量']/2
    elif (x['frequency']==3):
        if (x['小时_标签']>=8) & (x['小时_标签']<12):
            return x['日剂量']*2/3
        elif x['小时_标签']>=12:
            return x['日剂量']/3
    else:
        return x['日剂量']
    
# 小时标签用来 标记临时医嘱的时间，根据 frequency进行 日剂量的更改
# 例如： 一日三次， 则 按照 12点和6点划分， 12-6之间是 2/3 
drug_temp_result['小时_标签'] = drug_temp_result['start_datetime'].apply(lambda x: pd.to_datetime(x).hour)

drug_temp_result['日剂量_temp'] =drug_temp_result['日剂量'] 
drug_temp_result['日剂量'] = drug_temp_result.apply(dosage_temp,axis=1)
drug_temp_result = drug_temp_result.drop(columns='小时_标签')

In [134]:
print(drug_temp_result.shape)
print(drug_temp_result['patient_id'].nunique())
print(drug_temp_result['case_no'].nunique())

(133, 15)
74
93


In [135]:
# 保存处理临时医嘱
drug_temp_result=drug_temp_result.sort_values(['case_no','start_datetime']).reset_index(drop=True)
drug_temp_result.to_excel(project_path+'/data/processed_data/df_2.1.4.5_处理临时医嘱.xlsx')

In [136]:
drug_temp_result[drug_temp_result['case_no']==8785699]

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,用药上下午标签,日剂量_temp
103,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-15 15:24:52,2020-03-15 15:24:52,核实,2.5,2020-03-15,2020-03-15,1,5.0
104,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-18 09:01:31,2020-03-18 09:01:31,核实,5.0,2020-03-18,2020-03-18,0,5.0
105,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-24 22:26:14,2020-03-24 22:26:14,核实,2.5,2020-03-24,2020-03-24,1,5.0
106,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-27 10:20:47,2020-03-27 10:20:47,核实,5.0,2020-03-27,2020-03-27,0,5.0
107,8059463,8785699,0,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,2.5,2,2020-03-30 08:06:27,2020-03-30 08:06:27,核实,5.0,2020-03-30,2020-03-30,0,5.0


In [137]:
# 把‘临时医嘱’和‘长期医嘱’拼接
df_test_1 = pd.concat([drug_long,drug_temp_result],axis=0)
df_test_1 = df_test_1[drug_long.columns.tolist()]

# 拼接后重新排序
drug_vancomycin_4 = df_test_1.groupby(['case_no']).apply(
    lambda _df: _df.sort_values(by=['start_datetime'],ascending=1)).reset_index(drop=True)

In [138]:
drug_vancomycin_4.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0,2018-07-10,2018-07-16
1,3106885,4054467,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-08-03 09:56:12,2018-08-06 11:17:32,停止,2.0,2018-08-03,2018-08-06
2,3106885,4054467,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-08-06 11:17:32,2018-08-06 11:17:32,核实,2.0,2018-08-06,2018-08-06
3,7455055,4068971,0,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,1,2018-07-25 10:42:12,2018-07-25 10:42:12,核实,1.0,2018-07-25,2018-07-25
4,7455055,4068971,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,2.5,1,2018-07-25 18:30:46,2018-08-01 09:17:58,停止,2.5,2018-07-25,2018-08-01


#### 分裂日期

In [139]:
# 将时间拆开
def explode1(df, col):
    dftem1 = pd.DataFrame()
    for i in range(df.shape[0]):
        dftem = df.iloc[i].to_frame().T
        if isinstance(df.iloc[i][col], list):
            if len(df.iloc[i][col]) > 0:
                for j in range(len(df.iloc[i][col])):
                    dftem[col + "_"] = df.iloc[i][col][j]
                    dftem1 = pd.concat([dftem1, dftem])
            else:
                dftem1 = pd.concat([dftem1, dftem])
        else:
            dftem1 = pd.concat([dftem1, dftem])
    del dftem1[col]
    dftem1.rename(columns={col + "_": col}, inplace=True)
    return dftem1

In [140]:
drug_vancomycin_4.shape

(334, 13)

In [141]:
# 生成 分裂日期
drug_vancomycin_4['分裂日期'] = ''
for i in range(len(drug_vancomycin_4)):
    drug_vancomycin_4['分裂日期'][i] = list(pd.date_range(start=drug_vancomycin_4['start_date'][i], 
                                                      end=drug_vancomycin_4['end_date'][i]))

##　＝＝　把时间List拆开　＝＝    
drug_vancomycin_5 = explode1(drug_vancomycin_4,'分裂日期')
drug_vancomycin_5['分裂日期']=drug_vancomycin_5['分裂日期'].apply(lambda x: str(x).split(' ')[0])

drug_vancomycin_5['起始标签'] = drug_vancomycin_5.apply(lambda x: '起始' if x['分裂日期']==x['start_date'] else 
                                                                '结尾' if x['分裂日期']==x['end_date'] else 
                                                                '期间', axis=1)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [142]:
drug_vancomycin_5.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,分裂日期,起始标签
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-10,起始
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-11,期间
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-12,期间
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-13,期间
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-14,期间


In [143]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(1333, 15)
99
126


In [144]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.6_分裂日期.xlsx')

#### 打时间标签

In [145]:
# 打日期标签
def flag_ttt(x):
    if (x['起始标签']!='结尾'):
        if x['start_datetime'].hour<13:
            return 0
        else:
            return 1
    else:
        if x['end_datetime'].hour<13:
            return 0
        else:
            return 1

In [146]:
# 上午是0，下午是1
# 起始和期间看起始时间，结尾看停止时间
drug_vancomycin_5['start_datetime']=drug_vancomycin_5['start_datetime'].apply(lambda x: pd.to_datetime(x))
drug_vancomycin_5['end_datetime']=drug_vancomycin_5['end_datetime'].apply(lambda x: pd.to_datetime(x))

drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(lambda x: 0 if ((x['起始标签']!='结尾')and(x['start_datetime'].hour < 13)) else 1, axis=1)
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(flag_ttt,axis=1)

In [147]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(1333, 16)
99
126


In [148]:
drug_vancomycin_5.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,分裂日期,起始标签,时间标签
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-10,起始,1
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-11,期间,1
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-12,期间,1
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-13,期间,1
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2,2018-07-10,2018-07-16,2018-07-14,期间,1


In [149]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.7_打时间标签.xlsx')

#### 调整日剂量

In [150]:
temp_l = []

for i in drug_vancomycin_5.case_no.unique():
    temp = drug_vancomycin_5[drug_vancomycin_5['case_no']==i].reset_index(drop=True)

    for j in temp.分裂日期.unique():

        temp_2 = temp[temp['分裂日期']==j]
        if 0 in temp_2['long_d_order'].unique():
            temp_long = temp_2[temp_2['long_d_order']==1].tail(1).reset_index(drop=True)
            temp_st = temp_2[temp_2['long_d_order']==0]
            temp_st['日剂量'] = temp_st['日剂量'].sum()
            temp_st = temp_st.tail(1).reset_index(drop=True)
            # 同一天，既有长期医嘱又有短期医嘱
            if len(temp_long)>0:
                flag = temp_long.起始标签.unique()[0]
                if flag == '期间':
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] + temp_st.loc[0,'日剂量']                           
                    temp_l.append(temp_long)
                else:
                    temp_l.append(temp_st)
            # 该天全是短期医嘱
            else:
                temp_l.append(temp_st)
        # 该天全是长期医嘱
        else:
            temp_long = temp_2.tail(1).reset_index(drop=True)
            flag_l = temp_2.起始标签.unique().tolist()
            flag_time_l = temp_2.时间标签.unique().tolist()
            
            if ('结尾' in flag_l and '起始' in flag_l) or len(flag_time_l)>1:
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）                
                if (len(flag_time_l) == 1) and (temp_long.loc[0,'时间标签'] == 0):   
                    pass                                                         
                # 第一条结尾，第二条开始:   时间 上午 上午 取第二条(开始), 上下、下上、下下取mean
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）
                else:
                    temp_long.loc[0,'日剂量'] = temp_2.drop_duplicates(subset=['日剂量'],keep='last').日剂量.mean()
                temp_l.append(temp_long)
            else:
                flag = temp_long.loc[0,'起始标签']
                flag_time = temp_long.loc[0,'时间标签']
                # 开头：大于13点取半天，  结尾：小于13点取半天
                if flag=='起始' and flag_time==1:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2
                if flag=='结尾' and flag_time==0:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2

                temp_l.append(temp_long)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [151]:
df_Tcms_modification = pd.concat(temp_l,axis=0).reset_index(drop=True)
df_Tcms_modification['日剂量']=df_Tcms_modification['日剂量'].apply(lambda x: round(x,1))
del temp_l

In [152]:
# 设置patient_id，case_no字段数据数据格式，不然无法匹配id
df_Tcms_modification['patient_id']=df_Tcms_modification['patient_id'].astype('str')
df_Tcms_modification['case_no']=df_Tcms_modification['case_no'].astype('str')

In [153]:
print(df_Tcms_modification.shape)
print(df_Tcms_modification['patient_id'].nunique())
print(df_Tcms_modification['case_no'].nunique())

(1136, 16)
99
126


In [154]:
df_Tcms_modification.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,分裂日期,起始标签,时间标签
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,1.0,2018-07-10,2018-07-16,2018-07-10,起始,1
1,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0,2018-07-10,2018-07-16,2018-07-11,期间,1
2,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0,2018-07-10,2018-07-16,2018-07-12,期间,1
3,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0,2018-07-10,2018-07-16,2018-07-13,期间,1
4,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,2.0,2018-07-10,2018-07-16,2018-07-14,期间,1


In [155]:
df_Tcms_modification.to_excel(project_path+'/data/processed_data/df_2.1.4.8_调整日剂量.xlsx')

In [156]:
#### 用药稳态筛选

In [157]:
# df_Tcms_steady=df_Tcms_modification.copy()
# # 到稳态血药浓度（同一剂量至少连续服用3天)
# temp_list=[]
# for i in df_Tcms_steady['case_no'].unique():
#     print(i)
#     temp=df_Tcms_steady[df_Tcms_steady['case_no']==i]
#     temp=temp.reset_index(drop=True)
#     j=0
#     while j < temp.shape[0]:
# #         print(j)
#         # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
#         dosage_judge=temp.loc[j,'日剂量']    
#         # 1. 剂量判断
#         break_bool=False
#         for k in range(j+1,temp.shape[0]):
#             differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
#             if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
#                 # 判断是否中断
#                 break_bool=True
#                 break
#         # 是否是最后一天
#         if k != temp.shape[0]-1:
#             # 是否是稳态
#             if k-1-j>=2:
#                 temp_list.append(temp.loc[j:k-1,])
#             j=k
#             continue
#         else:
#             # 最后一天是否中断
#             if break_bool:
#                 # 是否是稳态
#                 if k-1-j>=2:
#                     temp_list.append(temp.loc[j:k-1,])
#             else:
#                 if k-j >=2:
#                     temp_list.append(temp.loc[j:k,])
#             j=k+1

# # list转换为DataFrame
# df_Tcms_steady = pd.concat(temp_list,axis=0).reset_index(drop=True)
# del temp_list

# print(df_Tcms_steady.shape)
# print(df_Tcms_steady['patient_id'].nunique())
# print(df_Tcms_steady['case_no'].nunique())

# df_Tcms_steady.head()

# df_Tcms_steady.to_excel(project_path+'/data/processed_data/df_2.1.4.7_用药稳态筛选.xlsx')

## 排除标准

In [158]:
# 数据严重缺失（如患者ID、主要研究结局等
df_Tcms_modification.isnull().any()

patient_id        False
case_no           False
long_d_order      False
drug_name         False
drug_spec         False
dosage            False
frequency         False
start_datetime    False
end_datetime      False
statusdesc        False
日剂量               False
start_date        False
end_date          False
分裂日期              False
起始标签              False
时间标签              False
dtype: bool

# 合并人口学特征

## 合并人口学信息

In [159]:
# 取 patient_id和case_no
df_popu=df_Tcms_modification[['patient_id','case_no']]

In [160]:
df_popu['patient_id']=df_popu['patient_id'].astype('int')
df_popu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  1136 non-null   int32 
 1   case_no     1136 non-null   object
dtypes: int32(1), object(1)
memory usage: 13.4+ KB


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [161]:
df_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8510 entries, 0 to 8509
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   patient_id  8510 non-null   int64
 1   gender      8510 non-null   int64
dtypes: int64(2)
memory usage: 133.1 KB


In [162]:
# merge gender
df_popu=pd.merge(df_popu,df_gender,on=['patient_id'],how='left')

In [163]:
df_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1759 entries, 0 to 1758
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   patient_id  1759 non-null   int64
 1   case_no     1759 non-null   int64
 2   age         1759 non-null   int64
dtypes: int64(3)
memory usage: 41.4 KB


In [164]:
# merge age
df_age_simple=df_age[['case_no','age']]
df_popu['case_no']=df_popu['case_no'].astype('int')
df_popu=pd.merge(df_popu,df_age_simple,on=['case_no'],how='left')

In [165]:
# merge height
df_height_simple=df_height[['case_no','record_content']]
df_height_simple=df_height_simple.rename(columns={'record_content':'身高'})
df_height_simple['身高']=df_height_simple['身高'].astype('float')
df_popu=pd.merge(df_popu,df_height_simple,on=['case_no'],how='left')

In [166]:
# merge weight
df_weight_simple=df_weight[['case_no','record_content']]
df_weight_simple=df_weight_simple.rename(columns={'record_content':'体重'})
df_weight_simple['体重']=df_weight_simple['体重'].astype('float')
df_popu=pd.merge(df_popu,df_weight_simple,on=['case_no'],how='left')

In [167]:
df_weight_simple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5016 entries, 69 to 342330
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   case_no  5016 non-null   int64  
 1   体重       5016 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 117.6 KB


In [168]:
df_popu.head()

,patient_id,case_no,gender,age,身高,体重
0,7406367,3964305,1,26,178.0,117.6
1,7406367,3964305,1,26,178.0,115.0
2,7406367,3964305,1,26,178.0,117.5
3,7406367,3964305,1,26,178.0,117.6
4,7406367,3964305,1,26,178.0,115.0


In [169]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(6070, 6)
99


In [170]:
# 删除重复数据
df_popu=df_popu.drop_duplicates(['patient_id','case_no','gender'])

In [171]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())
print(df_popu.case_no.nunique())

(126, 6)
99
126


In [172]:
df_popu.to_excel(project_path+'/data/processed_data/df_3.1_popu.xlsx')

## 计算BMI

In [173]:
BMI_list=[round(x*10000/y/y,2) for (x,y) in zip(df_popu['体重'].astype('float').tolist(),df_popu['身高'].astype('float').tolist())]

In [174]:
print(len(BMI_list))
print(df_popu.shape)

126
(126, 6)


In [175]:
df_popu['BMI']=np.array(BMI_list)

In [176]:
df_popu

,patient_id,case_no,gender,age,身高,体重,BMI
0,7406367,3964305,1,26,178.0,117.6,37.12
21,3106885,4054467,0,37,158.0,52.0,20.83
25,7455055,4068971,0,26,156.5,49.2,20.09
53,6453342,4225762,1,66,170.0,NaN,NaN
60,5695822,4301307,1,43,158.5,NaN,NaN
...,...,...,...,...,...,...,...
5982,8329779,9662993,0,47,156.5,57.2,23.35
5986,8325446,9669418,1,41,173.5,71.3,23.69
6007,5623241,9671945,1,50,177.0,62.0,19.79
6021,8059463,9745132,0,35,163.0,59.0,22.21


## 并入人口学特征

In [177]:
df_popu=df_popu[['case_no','gender','age','身高','体重','BMI']]

In [178]:
print(df_Tcms_modification.shape)
print(df_Tcms_modification['patient_id'].nunique())
print(df_Tcms_modification['case_no'].nunique())

(1136, 16)
99
126


In [179]:
df_Tcms_modification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   patient_id      1136 non-null   object        
 1   case_no         1136 non-null   object        
 2   long_d_order    1136 non-null   object        
 3   drug_name       1136 non-null   object        
 4   drug_spec       1136 non-null   object        
 5   dosage          1136 non-null   object        
 6   frequency       1136 non-null   object        
 7   start_datetime  1136 non-null   datetime64[ns]
 8   end_datetime    1136 non-null   datetime64[ns]
 9   statusdesc      1136 non-null   object        
 10  日剂量             1136 non-null   float64       
 11  start_date      1136 non-null   object        
 12  end_date        1136 non-null   object        
 13  分裂日期            1136 non-null   object        
 14  起始标签            1136 non-null   object        
 15  时间标签

In [180]:
df_Tcms_modification['case_no']=df_Tcms_modification['case_no'].astype('int')
df_Tcms_popu=pd.merge(df_Tcms_modification,df_popu,on=['case_no'],how='left')

In [181]:
print(df_Tcms_popu.shape)
print(df_Tcms_popu['patient_id'].nunique())
print(df_Tcms_popu['case_no'].nunique())

(1136, 21)
99
126


In [182]:
# 合并人口学特征
df_Tcms_popu.to_excel(project_path+'/data/processed_data/df_3.3_并入人口学特征.xlsx')

# 基础疾病

In [183]:
# 提取高血压、糖尿病、肝炎

In [184]:
df_Tcms_popu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1136 entries, 0 to 1135
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   patient_id      1136 non-null   object        
 1   case_no         1136 non-null   int32         
 2   long_d_order    1136 non-null   object        
 3   drug_name       1136 non-null   object        
 4   drug_spec       1136 non-null   object        
 5   dosage          1136 non-null   object        
 6   frequency       1136 non-null   object        
 7   start_datetime  1136 non-null   datetime64[ns]
 8   end_datetime    1136 non-null   datetime64[ns]
 9   statusdesc      1136 non-null   object        
 10  日剂量             1136 non-null   float64       
 11  start_date      1136 non-null   object        
 12  end_date        1136 non-null   object        
 13  分裂日期            1136 non-null   object        
 14  起始标签            1136 non-null   object        
 15  时间标签

In [185]:
# 高血压、糖尿病
df_Tcms_history=df_Tcms_popu.copy()
df_Tcms_history['patient_id']=df_Tcms_history['patient_id'].astype('int')
# 提取高血压
df_EH=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('高血压')]
# 提取糖尿病
df_DM=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('糖尿病')]
# 提取肝炎
df_HCA=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('肝炎')]

In [186]:
# df_basic_disease=pd.concat([df_EH,df_DM],axis=0)
# df_basic_disease=pd.concat([df_basic_disease,df_bomd],axis=0)
# df_basic_disease=pd.concat([df_basic_disease,df_HCA],axis=0)
# df_basic_disease=pd.concat([df_basic_disease,df_NEP],axis=0)

In [187]:
# 合并基础疾病
df_Tcms_history['高血压']=df_Tcms_history['patient_id'].apply(lambda x: 1 if x in df_EH.patient_id.unique() else 0)
df_Tcms_history['糖尿病']=df_Tcms_history['patient_id'].apply(lambda x: 1 if x in df_DM.patient_id.unique() else 0)
df_Tcms_history['肝炎']=df_Tcms_history['patient_id'].apply(lambda x: 1 if x in df_HCA.patient_id.unique() else 0)

In [188]:
print(df_Tcms_history.shape)
print(df_Tcms_history['patient_id'].nunique())
print(df_Tcms_history['case_no'].nunique())

(1136, 24)
99
126


In [189]:
df_Tcms_history.head()

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,起始标签,时间标签,gender,age,身高,体重,BMI,高血压,糖尿病,肝炎
0,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,...,起始,1,1,26,178.0,117.6,37.12,1,0,0
1,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,...,期间,1,1,26,178.0,117.6,37.12,1,0,0
2,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,...,期间,1,1,26,178.0,117.6,37.12,1,0,0
3,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,...,期间,1,1,26,178.0,117.6,37.12,1,0,0
4,7406367,3964305,1,他克莫司胶囊(普乐可复)[0.5mg*50粒],0.5mg*50粒,1,2,2018-07-10 21:44:10,2018-07-16 18:13:43,停止,...,期间,1,1,26,178.0,117.6,37.12,1,0,0


In [190]:
# 保存纳入标准数据
df_Tcms_history.to_excel(project_path+'/data/processed_data/df_4.3_并入基础疾病.xlsx')

# 并入手术时间

## 提取肾移植手术

In [191]:
df_surgical=pd.read_csv(project_path +'/data/raw_data/df_surgical.csv')
if 'Unnamed: 0' in df_surgical.columns:
    df_surgical = df_surgical.drop(['Unnamed: 0'], axis=1)

In [192]:
# 提取肾移植手术
df_surgical_kidney=df_surgical[df_surgical['surgery_name'].str.contains('肾') &
                              df_surgical['surgery_name'].str.contains('移植')]

In [193]:
print(df_surgical_kidney.shape)
print(df_surgical_kidney.patient_id.nunique())

(146, 15)
136


In [194]:
df_surgical_kidney.surgery_name.unique()

array(['同种异体肾移植术', '移植肾血管造影', '移植肾探查术', '再次同种异体肾移植术', '移植肾的肾动脉血管腔内成形术',
       '移植肾输尿管扩张术，经内窥镜', '移植肾切除术'], dtype=object)

In [195]:
df_surgical_kidney_group=df_surgical_kidney.groupby(['patient_id','surgery_date']).first().reset_index()
df_surgical_kidney_group=df_surgical_kidney_group[['patient_id','surgery_date']]

In [196]:
print(df_surgical_kidney_group.shape)
print(df_surgical_kidney_group.patient_id.nunique())

(119, 2)
118


In [197]:
df_surgical_kidney_group.isnull().any()

patient_id      False
surgery_date    False
dtype: bool

In [198]:
# 查看重复数据
df_surgical_kidney_group[df_surgical_kidney_group.duplicated(subset=['patient_id'],keep='first')]

,patient_id,surgery_date
39,7105834,2019-06-10 16:01:00


In [199]:
df_surgical_kidney.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 25 to 371
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   surgical_record_id      146 non-null    object 
 1   patient_id              146 non-null    int64  
 2   case_no                 146 non-null    int64  
 3   surgery_date            120 non-null    object 
 4   surgery_code            146 non-null    object 
 5   surgery_name            146 non-null    object 
 6   cut_grade               109 non-null    object 
 7   doctor_code             145 non-null    object 
 8   anesthesia_way          146 non-null    object 
 9   cut_healing             0 non-null      float64
 10  anesthesia_doctot_code  0 non-null      float64
 11  outp_no                 0 non-null      float64
 12  create_time             146 non-null    object 
 13  update_time             146 non-null    object 
 14  migrate_time            146 non-null    o

In [200]:
df_surgical_kidney_group.to_excel(project_path+'/data/processed_data/df_5.1_df_surgical_kidney_group.xlsx')

## 计算术后时间

In [201]:
## 并入肾移植手术
df_Tcms_surgery=pd.merge(df_Tcms_history,df_surgical_kidney_group,on=['patient_id'],how='left')

In [202]:
for i in range(df_Tcms_surgery.shape[0]):
    if pd.notnull(df_Tcms_surgery.loc[i,'surgery_date']):
        df_Tcms_surgery.loc[i,'术后时间']=(df_Tcms_surgery.loc[i,'start_datetime']-
                                       pd.to_datetime(df_Tcms_surgery.loc[i,'surgery_date'])).days +1

In [203]:
print(df_Tcms_surgery.shape)
print(df_Tcms_surgery.patient_id.nunique())

(1136, 26)
99


In [204]:
df_Tcms_surgery.to_excel(project_path+'/data/processed_data/df_5.2_Tcms_surgery.xlsx')

# 并入基因检测数据

## 提取他克莫司基因型

In [205]:
df_gene=pd.read_csv(project_path +'/data/raw_data/患者他克莫司基因-型.csv')
if 'Unnamed: 0' in df_gene.columns:
    df_gene = df_gene.drop(['Unnamed: 0'], axis=1)

In [206]:
# 提取患者的基因检测
df_gene_choose=df_gene[df_gene.patient_id.isin(df_Tcms_surgery.patient_id.unique())]
# 提取基因检测数据
df_gene_choose=df_gene_choose.groupby(['patient_id','test_result']).first().reset_index()
df_gene_choose=df_gene_choose[['patient_id','test_result']]

In [207]:
df_gene_choose.rename(columns={'test_result':'基因型'},inplace=True)

In [208]:
print(df_gene_choose.shape)
print(df_gene_choose.patient_id.nunique())

(8, 2)
8


In [209]:
df_gene_choose.head()

,patient_id,基因型
0,5623241,AG
1,7445835,AG
2,7522189,AG
3,8059463,GG
4,8141366,GG


## 并入基因数据

In [210]:
df_Tcms_gene=pd.merge(df_Tcms_surgery,df_gene_choose,on=['patient_id'],how='left')

In [211]:
print(df_Tcms_gene.shape)
print(df_Tcms_gene.patient_id.nunique())

(1136, 27)
99


In [212]:
df_Tcms_gene.to_excel(project_path+'/data/processed_data/df_6.2_Tcms_gene.xlsx')

# 添加他克莫司TDM检测

## 合并tdm检测数据

In [213]:
df_Tcms_gene.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1136 entries, 0 to 1135
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   patient_id      1136 non-null   int32         
 1   case_no         1136 non-null   int32         
 2   long_d_order    1136 non-null   object        
 3   drug_name       1136 non-null   object        
 4   drug_spec       1136 non-null   object        
 5   dosage          1136 non-null   object        
 6   frequency       1136 non-null   object        
 7   start_datetime  1136 non-null   datetime64[ns]
 8   end_datetime    1136 non-null   datetime64[ns]
 9   statusdesc      1136 non-null   object        
 10  日剂量             1136 non-null   float64       
 11  start_date      1136 non-null   object        
 12  end_date        1136 non-null   object        
 13  分裂日期            1136 non-null   object        
 14  起始标签            1136 non-null   object        
 15  时间标签

In [214]:
df_test_Tcms_choose['case_no']=df_test_Tcms_choose['case_no'].astype('int')
df_test_Tcms_choose.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   test_record_id  179 non-null    object 
 1   patient_id      179 non-null    int64  
 2   case_no         179 non-null    int32  
 3   test_date       179 non-null    object 
 4   project_name    179 non-null    object 
 5   test_result     179 non-null    float64
 6   result_unit     179 non-null    object 
 7   is_normal       179 non-null    object 
 8   refer_scope     0 non-null      object 
 9   synonym         179 non-null    object 
 10  tdm_date        179 non-null    object 
dtypes: float64(1), int32(1), int64(1), object(8)
memory usage: 14.8+ KB


In [215]:
import datetime
# 合并甲氨蝶呤tdm检测信息
# 所有tdm检测都认为是稳态检测。
temp_list=[]
for i in np.unique(df_Tcms_gene['case_no']):
    print(i)
    # 稳态 丙戊酸 temp
    temp_steady=df_Tcms_gene[df_Tcms_gene['case_no']==i]
    temp_steady=temp_steady.sort_values(['case_no','分裂日期'])
    temp_steady=temp_steady.reset_index(drop=True)
    
    # tdm temp
    temp_tdm=df_test_Tcms_choose[df_test_Tcms_choose['case_no']==i]
    temp_tdm=temp_tdm.sort_values(['case_no','test_date'])
    temp_tdm=temp_tdm.reset_index(drop=True)
    
    # tdm都认为是稳态浓度，不考虑3天稳态
    temp_steady['test_result']=np.nan
    for j in range(temp_tdm.shape[0]):
        # 提取tdm检测日期
        tdm_date=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])
#         print(tdm_date)
#         # 提取稳态前tdm前3天用药
#         temp_before_drug=temp_before[(pd.to_datetime(temp_before['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
#                                     (pd.to_datetime(temp_before['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 提取tdm前3天用药
        temp_steady_drug=temp_steady[(pd.to_datetime(temp_steady['分裂日期'])<=tdm_date-datetime.timedelta(days=1)) &
                                    (pd.to_datetime(temp_steady['分裂日期'])>=tdm_date-datetime.timedelta(days=4))]
        # 如果稳态前和稳态时用药有变化，则舍弃tdm数据，否则，并入到Tcms用药数据中
#         if temp_before_drug.shape[0] != temp_steady_drug.shape[0]:
#             continue
#         else:
        if temp_steady_drug.shape[0]>0:
            # 获取最近用药的索引
            index=temp_steady_drug.index.tolist()[-1]
            if pd.isnull(temp_steady.loc[index,'test_result']):
                temp_steady.loc[index,'tdm_date']=temp_tdm.loc[j,'tdm_date']
                temp_steady.loc[index,'test_result']=temp_tdm.loc[j,'test_result']
    temp_list.append(temp_steady)

3964305
4054467
4068971
4225762
4301307
4314349
4393323
4408901
4448130
4464857
4484442
4544201
4575222
4668171
4688574
4702803
4791498
4856583
4908430
4997217
5044376
5085040
5206958
5213070
5242966
5308143
5311344
5311448
5315580
5329337
5482630
5503283
5543879
5628819
5690359
5698218
5767306
5798617
5869316
5895566
5927217
5968061
5997487
6000880
6021971
6087548
6186748
6213888
6243347
6306839
6425502
6563920
6620389
6719091
6771809
6810989
6820316
6839844
6875625
6903583
6960521
7061897
7070027
7118876
7128646
7172608
7200062
7225186
7254193
7278432
7436362
7474224
7479148
7555600
7616911
7671502
7819335
7832149
7848888
7923612
8020263
8061910
8062256
8108818
8120695
8189109
8299769
8309978
8386426
8480233
8504098
8530515
8559368
8574883
8644797
8658250
8690248
8706614
8759451
8760180
8771059
8782274
8785699
8878109
8965655
8998595
9029107
9109601
9116374
9119891
9142222
9161107
9218290
9338958
9392071
9397853
9542118
9621029
9634093
9641782
9659240
9662993
9669418
9671945
9745132


In [216]:
temp_steady

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,体重,BMI,高血压,糖尿病,肝炎,surgery_date,术后时间,基因型,test_result,tdm_date
0,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
1,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
2,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
3,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
4,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
5,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
6,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
7,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
8,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,NaN,NaN
9,7445835,9812581,1,他克莫司胶囊(赛福开)[0.5mg*50粒],0.5mg*50粒,1,2,2020-07-31 10:54:04,2020-08-10 18:33:03,停止,...,45.4,20.07,0,0,0,NaN,NaN,AG,2.0,2020-08-10


In [217]:
temp_tdm

,test_record_id,patient_id,case_no,test_date,project_name,test_result,result_unit,is_normal,refer_scope,synonym,tdm_date
0,800008523628||Y001||1,7445835,9812581,2020-08-10 09:23:00,他克莫司药物浓度监测,2.0,ng/ml,N,NaN,FK506,2020-08-10


In [218]:
df_Tcms_tdm = pd.concat(temp_list,axis=0).reset_index(drop=True)
# df_Tcms_tdm = df_Tcms_tdm.drop_duplicates().reset_index(drop=True)
del temp_list

In [219]:
print(df_Tcms_tdm.shape)
print(df_Tcms_tdm['patient_id'].nunique())
print(df_Tcms_tdm['case_no'].nunique())

(1136, 29)
99
126


In [220]:
df_Tcms_tdm.columns

Index(['patient_id', 'case_no', 'long_d_order', 'drug_name', 'drug_spec',
       'dosage', 'frequency', 'start_datetime', 'end_datetime', 'statusdesc',
       '日剂量', 'start_date', 'end_date', '分裂日期', '起始标签', '时间标签', 'gender',
       'age', '身高', '体重', 'BMI', '高血压', '糖尿病', '肝炎', 'surgery_date', '术后时间',
       '基因型', 'test_result', 'tdm_date'],
      dtype='object')

In [221]:
# 保存添加tdm检测的甲氨蝶呤用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.1_添加tdm检测.xlsx')
df_Tcms_tdm.to_excel(writer)
writer.save()

## 删除tdm为空的数据

In [222]:
# 如果当前或前一条数据没有tdm，则删除
temp_list=[]
for i in df_Tcms_tdm.case_no.unique():
    temp=df_Tcms_tdm[df_Tcms_tdm['case_no']==i]
    temp=temp.sort_values(['case_no','分裂日期']).reset_index(drop=True)
    temp['target_日剂量']=np.nan
    # 打个标签
    # 他克莫司半衰期最长43小时
    for j in range(temp.shape[0]-1):
        if ((pd.notnull(temp.loc[j,'test_result']) & pd.isnull(temp.loc[j+1,'test_result'])) &
            ((pd.to_datetime(temp.loc[j,'分裂日期'])+datetime.timedelta(days=2))>=pd.to_datetime(temp.loc[j+1,'分裂日期']))):
            temp.loc[j,'target_日剂量']=temp.loc[j+1,'日剂量']
    # 提取有效用药数据
    temp_choose=temp[temp['target_日剂量'].notnull()]
    temp_list.append(temp_choose)

In [223]:
# 排序
df_Tcms_tdm_choose=pd.concat(temp_list,axis=0)
del temp_list
df_Tcms_tdm_choose=df_Tcms_tdm_choose.sort_values(['case_no','分裂日期']).reset_index(drop=True)

In [224]:
print(df_Tcms_tdm_choose.shape)
print(df_Tcms_tdm_choose['patient_id'].nunique())
print(df_Tcms_tdm_choose['case_no'].nunique())

(156, 30)
86
109


In [225]:
# 保存tdm非空的用药
df_Tcms_tdm_choose.to_excel(project_path+'/data/processed_data/df_7.2_Tcms_tdm_dosage_choose.xlsx')

## 统计前后剂量差

In [226]:
# 统计前后剂量差是否过大
for j in range(df_Tcms_tdm_choose.shape[0]):
    df_Tcms_tdm_choose.loc[j,'前后剂量差']=df_Tcms_tdm_choose.loc[j,'target_日剂量']-df_Tcms_tdm_choose.loc[j,'日剂量']

In [227]:
print(df_Tcms_tdm_choose.shape)
print(df_Tcms_tdm_choose['patient_id'].nunique())
print(df_Tcms_tdm_choose['case_no'].nunique())

(156, 31)
86
109


In [228]:
# 保存tdm非空的用药
df_Tcms_tdm_choose.to_excel(project_path+'/data/processed_data/df_7.3_统计前后剂量差.xlsx')

In [229]:
df_Tcms_tdm_choose['target_日剂量'].value_counts()

2.0    35
3.0    23
1.0    22
1.5    19
4.0    17
2.5    11
0.5     8
5.0     8
4.5     3
5.5     2
6.0     2
3.5     2
4.2     1
1.8     1
1.2     1
3.2     1
Name: target_日剂量, dtype: int64

# 增加联合用药

## 提取联合用药

In [230]:
# 提取患者的联合用药
df_drug_other=df_doctor_order[~ df_doctor_order['drug_name'].str.contains('他克莫司|葡萄糖|氯化钠')]
# 患者限定
df_drug_other=df_drug_other[df_drug_other['case_no'].isin(df_Tcms_tdm_choose['case_no'].unique())]
df_drug_other=df_drug_other.reset_index(drop=True)

In [231]:
print(df_drug_other.shape)
print(df_drug_other['patient_id'].nunique())

(33899, 10)
86


In [232]:
df_drug_other

,patient_id,case_no,long_d_order,drug_name,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,7406367,3964305,0,"血脂四项（CHO,TG,HDL-C,LDL-C）",NaN,NaN,ONCE,2018-07-10 17:05:05,NaN,执行
1,7406367,3964305,0,梅毒螺旋体特异抗体(电化学发光法),NaN,NaN,ONCE,2018-07-10 17:05:05,NaN,执行
2,7406367,3964305,0,静脉输液(其它药物集中配置),NaN,NaN,ONCE,2018-07-13 12:15:13,NaN,核实
3,7406367,3964305,0,一次性无菌注射器（20ML),NaN,NaN,ONCE,2018-07-13 12:15:13,NaN,核实
4,7406367,3964305,0,呋塞米注射液(南国)[2ml*1支],2ml*1支,40mg,ONCE,2018-07-13 12:15:13,NaN,核实
...,...,...,...,...,...,...,...,...,...,...
33894,7445835,9812581,0,静脉输液(住院),NaN,NaN,ONCE,2020-07-27 17:27:02,NaN,核实
33895,7445835,9812581,0,一次性使用精密过滤输液器南京IS-JL-5.0,NaN,NaN,ONCE,2020-07-27 17:27:02,NaN,核实
33896,7445835,9812581,0,会诊申请,NaN,NaN,NaN,2020-07-27 17:30:25,NaN,核实
33897,7445835,9812581,0,预冲式冲管注射器(10ml)(威高),NaN,NaN,ONCE,2020-07-28 08:37:08,NaN,核实


In [233]:
df_drug_other.to_excel(project_path+'/data/processed_data/df_8.1_drug_other.xlsx')

## 统计联合用药频数

In [234]:
# 统计用药频数，由业务人员确定选用哪些联合用药。
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_Tcms_drug_other=df_Tcms_tdm_choose.copy()
df_Tcms_drug_other['case_no']=df_Tcms_drug_other['case_no'].astype('int')
temp_list=[]
for i in range(df_Tcms_drug_other.shape[0]):
    case_no=df_Tcms_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 时间限定：用药前-tdm检测前
    drug_time=df_Tcms_drug_other.loc[i,'分裂日期']
    tdm_time=df_Tcms_drug_other.loc[i,'tdm_date']
    # 提取用药前3天内-tdm前的联合用药。
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(tdm_time)-datetime.timedelta(days=4)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    temp_list.append(drug_other_1)

df_drug_other_choose = pd.concat(temp_list,axis=0)
del temp_list
df_drug_other_choose=df_drug_other_choose.drop_duplicates().reset_index(drop=True)
drug_name_counts=df_drug_other_choose.drug_name.value_counts()
df_drug_name_counts=pd.DataFrame(data={'药物名称':drug_name_counts.index,
                                     '频次':drug_name_counts.values})
print(df_drug_name_counts.shape)
# 保存联合用药名
df_drug_name_counts.to_excel(project_path+'/data/processed_data/df_8.2_统计联合用药频数.xlsx')

(390, 2)


## 合并联合用药

In [235]:
# 业务人员给定的联合用药
tang=['地塞米松','甲泼尼龙','泼尼松','可的松']
zhizi=['奥美拉唑','泮托拉唑','艾普拉唑','雷贝拉唑','兰索拉唑','雷尼替丁']
gai=['硝苯地平','氨氯地平','尼群地平','非洛地平','地尔硫卓']
other_immu=['环孢素','吗替麦考酚酯','环磷酰胺','硫唑嘌呤','甲氨蝶呤']
antibiotic=['克拉霉素','阿奇霉素']

In [236]:
df_tang=df_drug_other[df_drug_other.drug_name.str.contains('地塞米松|甲泼尼龙|泼尼松|可的松')]
df_tang.to_excel(project_path+'/data/processed_data/df_tang.xlsx')

In [237]:
df_zhizi=df_drug_other[df_drug_other.drug_name.str.contains('奥美拉唑|泮托拉唑|艾普拉唑|雷贝拉唑|兰索拉唑|雷尼替丁')]
df_zhizi.to_excel(project_path+'/data/processed_data/df_zhizi.xlsx')

In [238]:
df_gai=df_drug_other[df_drug_other.drug_name.str.contains('硝苯地平|氨氯地平|尼群地平|非洛地平|地尔硫卓')]
df_gai.to_excel(project_path+'/data/processed_data/df_gai.xlsx')

In [239]:
df_other_immu=df_drug_other[df_drug_other.drug_name.str.contains('环孢素|吗替麦考酚酯|环磷酰胺|硫唑嘌呤|甲氨蝶呤')]
df_other_immu.to_excel(project_path+'/data/processed_data/df_other_immu.xlsx')

In [263]:
df_behavior_therapy=df_drug_other[df_drug_other.drug_name.str.contains('克拉霉素|阿奇霉素')]
df_behavior_therapy.to_excel(project_path+'/data/processed_data/df_behavior_therapy.xlsx')

In [264]:
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_Tcms_drug_other=df_Tcms_tdm_choose.copy()
df_Tcms_drug_other=df_Tcms_drug_other.reset_index(drop=True)
for i in range(df_Tcms_drug_other.shape[0]):
    case_no=df_Tcms_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
    drug_time=df_Tcms_drug_other.loc[i,'分裂日期']
    tdm_time=df_Tcms_drug_other.loc[i,'tdm_date']
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(tdm_time)-datetime.timedelta(days=4)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    # 联合用药列表
    drug_other_list=drug_other_1.drug_name.unique()
    # 联合用药转置
    drug_name=['糖皮质激素','质子泵抑制剂','钙离子阻抗剂','其他免疫抑制剂','克拉霉素']
    drug_name_list=[tang,zhizi,gai,other_immu,antibiotic]
    for name,name_list in zip(drug_name,drug_name_list):
        match_value=[1 if re.match(x,y) else 0 for x in name_list for y in drug_other_list]
        if np.sum(match_value)>0:
            df_Tcms_drug_other.loc[i,name]=1
        else:
            df_Tcms_drug_other.loc[i,name]=0

In [265]:
print(df_Tcms_drug_other.shape)
print(df_Tcms_drug_other['patient_id'].nunique())

(156, 36)
86


In [266]:
# 时间限定的联合用药
df_Tcms_drug_other.to_excel(project_path+'/data/processed_data/df_8.3_合并联合用药.xlsx')

# 合并其他相关检测

## 提取其他检测

In [267]:
# 提取甲氨蝶呤患者的所有检测
df_test_other=df_test[df_test['case_no'].isin(df_Tcms_drug_other['case_no'].unique())]

In [268]:
# 提取患者的其他检测
df_test_other = df_test_other[~ df_test_other['project_name'].str.contains('他克莫司')]
df_test_other = df_test_other.reset_index(drop=True)

In [269]:
df_test_other.isnull().any()

test_record_id    False
patient_id        False
case_no           False
test_date         False
project_name      False
test_result       False
result_unit        True
is_normal          True
refer_scope        True
synonym            True
dtype: bool

In [270]:
print(df_test_other.shape)
print(df_test_other['patient_id'].nunique())

(43202, 10)
86


In [271]:
# 保存其他检测
df_test_other.to_csv(project_path+'/data/processed_data/df_9.1_test_other.csv')

## 统计其他检测项频数

In [272]:
df_test_other_name=pd.DataFrame(df_test_other['project_name'].value_counts())

In [273]:
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.2_统计其他检测项频数.xlsx')
df_test_other_name.to_excel(writer)
writer.save()

## 合并其他检测

In [274]:
def judge_float(x):
    try:
        a=float(x)
        return a
    except:
        return np.NaN

In [275]:
df_Tcms_test_other=df_Tcms_drug_other.copy()
# 将用药后不良反应发生前的最近一次其他检测转置到一行中
for i in range(df_Tcms_test_other.shape[0]):
    case_no=df_Tcms_test_other.loc[i,'case_no']
    drug_time=df_Tcms_test_other.loc[i,'分裂日期']
    tdm_date=df_Tcms_test_other.loc[i,'tdm_date']
    # case_no的其他检测
    test_other= df_test_other[df_test_other['case_no']==case_no]
    test_other=test_other.reset_index(drop=True)
    test_other['test_date']=test_other['test_date'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂检测日期
    # 取用药后-不良反应发生前的其他检测，此案例中start_datetime>tdm_date-1。
    test_other_1=test_other[(test_other['test_date']>=pd.to_datetime(tdm_date)-datetime.timedelta(days=4))&
                            (test_other['test_date']<=tdm_date)]
    # 取最近的一次其他检测
    test_other_1=test_other_1.sort_values(['test_date'])
    test_other_1=test_other_1.drop_duplicates(['project_name'],keep='last')
    test_other_1=test_other_1.reset_index(drop=True)
    
    # 检验项转置为表头
    if test_other_1.shape[0]>0:
        test_other_1['test_result']=test_other_1['test_result'].apply(judge_float)
        test_other_pivot=test_other_1.pivot_table('test_result',['case_no'], 'project_name')
        test_other_pivot=test_other_pivot.reset_index(drop=True)
        for j in test_other_pivot.columns[1:]:
            df_Tcms_test_other.loc[i,j]=test_other_pivot.loc[0,j]
#     break
#     if case_no == '410308':
#         print(temp.loc[0,'start_datetime'])
#         print(temp.loc[0,'bmd_time'])
#         print(test_other_1)
#         print(test_other_2[test_other_2['project_name']=='肌酐(干式)'])
#         break

In [276]:
print(df_Tcms_test_other.shape)
print(df_Tcms_test_other['patient_id'].nunique())

(156, 405)
86


In [277]:
# 合并其他检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.3_合并其他检测.xlsx')
df_Tcms_test_other.to_excel(writer)
writer.save()

## 删除缺失率超过50%的列

In [278]:
df_Tcms_test_other.columns

Index(['patient_id', 'case_no', 'long_d_order', 'drug_name', 'drug_spec',
       'dosage', 'frequency', 'start_datetime', 'end_datetime', 'statusdesc',
       ...
       '抗β2糖蛋白I型IgA抗体定量', '抗β2糖蛋白I型IgG抗体定量', '抗β2糖蛋白I型IgM抗体定量', 'JC病毒定量',
       '多瘤病毒定量', '超敏C反应蛋白', '-MicroR', '同型半胱氨酸', '24h尿微量白蛋白测定',
       '乙肝病毒定量（FQ-HBV-DNA）'],
      dtype='object', length=405)

In [279]:
# 删除缺失超过50%的检测
for i in df_Tcms_test_other.columns:
    x=df_Tcms_test_other[i].isnull().sum()
    per_x = x/df_Tcms_test_other.shape[0]  
    if per_x >= 0.5:
        print(i)
        del df_Tcms_test_other[i]

surgery_date
术后时间
基因型
促甲状腺素
免疫球蛋白A
免疫球蛋白G
免疫球蛋白M
尿a1-微球蛋白测定
尿a2-巨球蛋白测定
尿β2微球蛋白测定
尿免疫球蛋白定量测定
尿微量白蛋白测定
尿渗量
尿红细胞位相
尿转铁蛋白测定
尿轻链Kappa定量
尿轻链Lambda定量
形态分析：均一型
形态分析：多形型
游离T3
游离T4
甲状三碘原氨酸
甲状腺素
皮质醇(日0时)
皮质醇(次日8时)
葡萄糖(2H)
血清免疫球蛋白G4测定
血清总补体
补体成份3
补体成份4
24h尿量
尿总蛋白浓度
Hb A2
异丙醇试验
热溶血试验
直接抗人球蛋白试验
红细胞包涵体检查
红细胞渗透脆性试验
葡萄糖6-磷酸脱氢酶活性
蔗糖溶血试验
血清酸化溶血试验
间接抗人球蛋白试验
C反应蛋白
EB病毒定量
丙肝抗体
乙肝病毒E抗体
乙肝病毒E抗原
乙肝病毒核心抗体
乙肝病毒表面抗体
乙肝病毒表面抗原（定性）
前-脑利尿肽
巨细胞病毒定量
总蛋白浓度
抗中性粒细胞胞浆抗体
梅毒螺旋体特异性抗体[电化学发光法]
淋巴细胞培养+干扰素(基础水平N)
淋巴细胞培养+干扰素（刺激水平T）
真菌D-葡聚糖（1-3）-β-D
糖化血红蛋白
红细胞沉降率测定
结核杆菌γ-干扰素释放试验(T-N)
铁蛋白
降钙素原
A.GD
A1.GI
B.GD
B.GI
C.F
CTL.GD
D.RH
E.F
I.IAC
II.IAC
III.IAC
不饱和铁结合力
丙型肝炎病毒抗体[电化学发光]
中荧光比值网织红细胞
乙肝病毒表面抗原
二氧化碳分压
低荧光比值网织红细胞
修正二氧化碳分压
修正氧分压
修正酸碱度
动脉血红蛋白浓度
可溶性转铁蛋白受体
实际碳酸氢根
小动脉氧含量
幼稚网织红细胞
当前大气压
总铁结合力
抗RNP抗体定量
抗Sm抗体定量
抗体筛查试验
抗双链DNA抗体测定
抗心磷脂抗体IgG
抗心磷脂抗体IgM
抗核抗体测定
抗磷脂酶A2受体抗体IgG检测
标准碳酸氢根
氧分压
氧合血红蛋白百分率
氧饱和度
狼疮初筛/狼疮确认
狼疮抗凝物质初筛试验
狼疮抗凝物质确认试验
病人体温
碳氧合血红蛋白百分率
细胞内碱剩余
细胞外碱剩余
网织红细胞百分比
网织红细胞计数
肺泡气氧分压
艾滋病病毒抗体
转铁蛋白
转铁蛋白饱和度
还原血红蛋白百分率
铁离子
霉酚酸药物浓度监测
高荧光比值网织红细胞
高铁血

In [280]:
print(df_Tcms_test_other.shape)
print(df_Tcms_test_other['patient_id'].nunique())

(156, 106)
86


In [281]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.4_删除缺失超过50%的列.xlsx')
df_Tcms_test_other.to_excel(writer)
writer.save()

In [282]:
df_Tcms_test_other.target_日剂量.value_counts()

2.0    35
3.0    23
1.0    22
1.5    19
4.0    17
2.5    11
0.5     8
5.0     8
4.5     3
5.5     2
6.0     2
3.5     2
4.2     1
1.8     1
1.2     1
3.2     1
Name: target_日剂量, dtype: int64

## 人工删除没有意义的指标

In [283]:
del df_Tcms_test_other['国际标准比率']
del df_Tcms_test_other['尿管型(仪器定量)']
del df_Tcms_test_other['尿酵母菌']
del df_Tcms_test_other['总二氧化碳']

del df_Tcms_test_other['无机磷']
del df_Tcms_test_other['总钙']
del df_Tcms_test_other['氯离子']
del df_Tcms_test_other['葡萄糖']
del df_Tcms_test_other['钠离子']
del df_Tcms_test_other['钾离子']
del df_Tcms_test_other['镁离子']

In [284]:
df_Tcms_test_other.to_excel(project_path+'/data/processed_data/df_9.5_人工删除没有意义的指标.xlsx')

In [285]:
df_Tcms_test_other.target_日剂量.value_counts()

2.0    35
3.0    23
1.0    22
1.5    19
4.0    17
2.5    11
0.5     8
5.0     8
4.5     3
5.5     2
6.0     2
3.5     2
4.2     1
1.8     1
1.2     1
3.2     1
Name: target_日剂量, dtype: int64